<a href="https://colab.research.google.com/github/jgtabet02123/livedrop-JeanPaulTabet/blob/main/livedrop-JeanPaulTabet/docs/notebooks" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Shoplite RAG Deployment



In [1]:
!pip install -q torch transformers sentence-transformers faiss-cpu flask pyngrok pyyaml bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 18.0 MB/s eta 0:00:00


# Imports

In [2]:
import os
import json
import threading
import time
import requests

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from flask import Flask, request, jsonify
from pyngrok import ngrok

# Knowledge Base


In [3]:

KNOWLEDGE_BASE = [
    {
        "id": "doc1",
        "title": "Shoplite User Registration Process",
        "content": """To create a Shoplite account, users begin by visiting the registration page on the website or mobile app. They must provide a valid email address, a secure password, and basic profile details including full name and phone number. After submitting the form, Shoplite sends a verification email which must be confirmed within 24 hours.

Shoplite offers two types of accounts: buyer accounts and seller accounts. Buyer accounts are free and allow customers to browse, purchase products, and leave reviews. Seller accounts require additional business information, such as tax identification and proof of business registration, which undergoes manual verification. This process usually takes 2–3 business days.

For security purposes, Shoplite enforces password strength rules and supports optional two-factor authentication (2FA). Users who fail to verify their email cannot access core platform features. Duplicate accounts with the same email are not allowed, and Shoplite reserves the right to suspend accounts that provide false information.

Overall, the registration process is designed to balance convenience for buyers with compliance and fraud prevention for sellers."""
    },
    {
        "id": "doc2",
        "title": "Shoplite Shopping Cart Features",
        "content": """The Shoplite shopping cart allows customers to conveniently manage items they intend to purchase. Users can add multiple products from different sellers, view product details, and adjust quantities directly from the cart. For registered users, cart contents are automatically saved across devices and browsing sessions. Guest users’ carts persist only during the active session.

The cart interface supports promotional code entry, which automatically applies discounts during checkout if the code is valid. Customers may also save items for later or move them to a wishlist. Subtotals and estimated shipping costs are displayed in real time, helping buyers manage expenses before committing to purchase.

Shoplite also alerts customers about stock availability. If an item in the cart goes out of stock, the system displays a notification. For sellers, cart analytics provide insights into abandoned carts, enabling targeted promotions.

Together, these features ensure a smooth and transparent shopping experience, while encouraging conversions through reminders and personalized offers."""
    },
    {
        "id": "doc3",
        "title": "Shoplite Product Search and Filtering",
        "content": """Shoplite provides a robust product search engine designed to help users quickly find relevant items. Customers can search by product name, category, brand, or seller. Autocomplete suggestions appear as users type, showing trending searches and popular products. Advanced filtering allows customers to narrow results by price range, availability, location, rating, and shipping speed.

Search results are ranked based on a relevance algorithm that considers keyword match, product popularity, and customer reviews. Sponsored products are clearly labeled and displayed in prominent positions. To enhance discoverability, sellers are encouraged to use accurate titles, detailed descriptions, and appropriate tags.

Filters are especially useful for high-volume categories such as electronics and fashion. Customers can apply multiple filters simultaneously, and results update dynamically without requiring a full page reload. The platform also supports sorting by “best match,” “price: low to high,” “price: high to low,” and “new arrivals.”

Overall, the Shoplite search and filtering system ensures users can find what they need with minimal friction, improving the likelihood of successful transactions."""
    },
    {
        "id": "doc4",
        "title": "Shoplite Checkout Process",
        "content": """The checkout process on Shoplite is designed for simplicity and security. Customers begin by reviewing their shopping cart, applying discount codes, and confirming item quantities. Once satisfied, they proceed to checkout, where they provide shipping information and choose a delivery option.

Shoplite offers both guest checkout and registered user checkout. Registered users benefit from saved addresses, faster payment processing, and order history tracking. During checkout, buyers can select from available payment methods, including credit cards, debit cards, digital wallets, and Shoplite’s internal wallet. All transactions are secured with SSL encryption.

Customers receive a final order summary that details the items purchased, applied discounts, shipping costs, and estimated delivery times. They must confirm this information before payment is processed. After successful payment, Shoplite generates an order confirmation page and sends an email with tracking details.

The streamlined checkout process minimizes cart abandonment and provides customers with transparency at every step."""
    },
    {
        "id": "doc5",
        "title": "Shoplite Payment Methods and Security",
        "content": """Shoplite supports multiple payment methods to ensure flexibility and convenience. Buyers can pay using major credit cards, debit cards, PayPal, Apple Pay, Google Pay, and Shoplite Wallet. For select regions, cash on delivery is also available. Sellers are paid out through bank transfers or supported e-wallets, depending on their location.

To maintain high security standards, all payments are processed through PCI-DSS-compliant gateways. Shoplite uses encryption, tokenization, and fraud detection algorithms to safeguard transactions. Suspicious activity triggers additional verification, such as one-time passwords or manual review.

Shoplite also employs 3D Secure for card transactions, adding an extra authentication step for high-value purchases. Sensitive payment data is never stored directly on Shoplite’s servers, reducing exposure to breaches.

Customers are encouraged to enable two-factor authentication on their accounts for additional protection. Shoplite regularly audits its payment systems to ensure compliance with international financial regulations."""
    },
  {
        "id": "doc6",
        "title": "Shoplite Order Tracking and Delivery",
        "content": """Once an order is placed, Shoplite provides real-time tracking capabilities. Customers receive a tracking number linked to the courier’s system. Tracking information includes order confirmation, dispatch details, estimated delivery date, and real-time location updates when available.

Shoplite partners with multiple logistics providers to ensure broad delivery coverage. Customers can choose between standard, expedited, and same-day delivery options depending on availability. Each option displays estimated costs and delivery times at checkout.

The platform also allows customers to update delivery preferences, such as changing addresses or scheduling a specific delivery time, depending on courier support. Push notifications and email updates keep buyers informed about order progress.

Delivery issues, such as delays or damaged goods, are handled through Shoplite’s customer support. Buyers can request compensation or replacement, and the platform mediates disputes with logistics partners when necessary."""
    },
     {
        "id": "doc7",
        "title": "Shoplite Return and Refund Policies",
        "content": """Shoplite offers a standardized return and refund policy to maintain trust between buyers and sellers. Most products are eligible for return within a **30-day window** from delivery, provided they are unused and in original packaging. Certain categories, such as perishables and personal care items, are exempt.

To initiate a return, customers must log into their account, navigate to the order details page, and request a return authorization. Shoplite generates a prepaid return shipping label for eligible items. Sellers are notified immediately and must approve the request within 48 hours.

Refunds are processed after the returned item is inspected. Buyers typically receive their refund within 5–7 business days through the original payment method. For partial refunds, Shoplite ensures transparent communication about deductions such as restocking fees.

By combining clear policies with automated workflows, Shoplite minimizes disputes and ensures a fair process for both buyers and sellers."""
    },
      {
        "id": "doc8",
        "title": "Shoplite Product Reviews and Ratings",
        "content": """Customer feedback is a key part of the Shoplite experience. Buyers can leave reviews and ratings for products they have purchased. Ratings are given on a five-star scale, while reviews allow for written comments and optional images. Verified purchase labels distinguish genuine reviews from unverified ones.

The review system uses moderation filters to detect inappropriate content, spam, or fake reviews. Customers can also report suspicious feedback, which triggers manual review. Shoplite prioritizes transparency by showing both positive and negative reviews, ensuring customers can make informed decisions.

Sellers are encouraged to respond to reviews to demonstrate customer engagement. High ratings improve product visibility in search results, while consistently low ratings may lead to product removal or account penalties.

Reviews play an important role in building trust, influencing buying decisions, and shaping seller reputation on the platform."""
    },
    {
        "id": "doc9",
        "title": "Shoplite Seller Account Setup and Management",
        "content": """Sellers who want to join Shoplite must first create a seller account by visiting the dedicated registration page. They are required to submit business information, such as company name, tax identification, and proof of registration. Shoplite verifies these documents within 2–3 business days.

Once approved, sellers can access the Seller Dashboard, which includes tools for product uploads, inventory tracking, and sales analytics. They can also manage orders, returns, and customer messages directly from the dashboard. Shoplite provides training materials and best-practice guides to help sellers succeed.

Sellers must comply with Shoplite’s marketplace policies, which include accurate product descriptions, timely shipping, and responsible customer service. Accounts that violate these policies may be suspended or permanently banned.

By offering a transparent and supportive setup process, Shoplite ensures sellers can efficiently manage their business operations and grow their online presence.
"""
    },
    {
        "id": "doc10",
        "title": "Shoplite Inventory Management Tools",
        "content": """Shoplite provides sellers with powerful inventory management features accessible through the Seller Dashboard. Sellers can bulk upload product listings using CSV templates or integrate directly with third-party inventory systems via the Shoplite API.

The platform automatically updates inventory levels as sales occur. Low-stock alerts notify sellers when items fall below predefined thresholds, reducing the risk of overselling. Sellers can also track inventory turnover, identify slow-moving products, and analyze seasonal demand trends.

Shoplite supports multi-location inventory management, allowing sellers to track stock across warehouses or retail outlets. For large sellers, real-time synchronization ensures consistency between physical and online sales.

These tools are designed to give sellers full visibility and control, ultimately helping them optimize operations and improve customer satisfaction.
"""
    },
     {
        "id": "doc11",
        "title": "Shoplite Commission and Fee Structure",
        "content": """Shoplite’s business model is based on commissions and service fees. For most product categories, Shoplite charges a **10% commission** on completed sales. High-margin categories like electronics may incur higher fees, while certain categories such as books and handmade crafts may qualify for reduced rates.

In addition to commissions, sellers pay small transaction processing fees that vary by payment method. Optional services, such as sponsored product ads and premium storefront customization, incur additional charges. Shoplite provides sellers with a monthly invoice detailing all fees and deductions.

To encourage growth, Shoplite occasionally offers reduced commissions for new sellers during their first three months. Failure to comply with platform rules may result in penalties, including higher fees or account suspension.

By maintaining a transparent fee structure, Shoplite ensures fairness while supporting its operational costs.
"""
    },
    {
        "id": "doc12",
        "title": "Shoplite Customer Support Procedures",
        "content": """Shoplite provides multiple channels for customer support, including live chat, email, and a help center with FAQs. Customers can access support directly from the website or mobile app. For urgent issues such as payment failures or undelivered orders, Shoplite prioritizes live chat and phone support.

Support tickets are categorized by issue type (billing, delivery, returns, account issues) and routed to specialized agents. Shoplite aims to resolve most cases within 48 hours. Complex disputes between buyers and sellers may take longer, requiring investigation.

An AI-powered chatbot handles common inquiries, such as password resets and order tracking, freeing human agents to focus on complex problems. Shoplite continuously trains its support team to ensure consistent, empathetic, and efficient service.

This multi-layered approach ensures customers receive timely and accurate support regardless of their issue.
"""
    },

    {
        "id": "doc13",
        "title": "Shoplite Mobile App Features",
        "content": """The Shoplite mobile app offers a seamless shopping experience optimized for smartphones and tablets. Features include personalized home feeds, voice search, barcode scanning, and push notifications for deals or order updates.

The app integrates with device-level biometric authentication, allowing for secure logins and one-touch payments. Offline browsing enables users to view cached products and wishlists even without internet access, with updates syncing once connectivity is restored.

Sellers can also use the app to manage their accounts, upload product photos, and respond to customer inquiries on the go. The app receives frequent updates to enhance performance and security.

By focusing on convenience, the Shoplite mobile app strengthens customer engagement and supports sellers in managing their businesses from anywhere.
"""
    },

    {
        "id": "doc14",
        "title": "Shoplite Mobile App Features",
        "content": """The Shoplite mobile app offers a seamless shopping experience optimized for smartphones and tablets. Features include personalized home feeds, voice search, barcode scanning, and push notifications for deals or order updates.

The app integrates with device-level biometric authentication, allowing for secure logins and one-touch payments. Offline browsing enables users to view cached products and wishlists even without internet access, with updates syncing once connectivity is restored.

Sellers can also use the app to manage their accounts, upload product photos, and respond to customer inquiries on the go. The app receives frequent updates to enhance performance and security.

By focusing on convenience, the Shoplite mobile app strengthens customer engagement and supports sellers in managing their businesses from anywhere.
"""
    },

    {
        "id": "doc15",
        "title": "Shoplite Security and Privacy Policies",
        "content": """Shoplite places strong emphasis on security and user privacy. All data transmissions are encrypted using TLS protocols, and sensitive information such as passwords and payment tokens are hashed or tokenized. Shoplite follows GDPR and CCPA regulations, offering users full control over their personal data.

Customers can download or delete their data through account settings. Shoplite commits to never selling personal data to third parties. User behavior may be analyzed internally for personalization and fraud prevention, but this data is anonymized where possible.

Regular security audits, penetration tests, and bug bounty programs ensure continuous monitoring. In the event of a data breach, Shoplite follows strict notification protocols, informing users within 72 hours as required by law.

By prioritizing security and privacy, Shoplite builds trust and ensures compliance with international standards.
"""
    },

     {
        "id": "doc16",
        "title": "Shoplite Promotional Codes and Discounts",
        "content": """Shoplite offers promotional codes and discounts to attract and retain customers. These codes may apply site-wide, to specific categories, or individual products. Customers enter codes during checkout, and valid discounts are automatically reflected in the order summary.

Types of promotions include percentage discounts, fixed-amount discounts, free shipping, and buy-one-get-one (BOGO) offers. Sellers can also create their own promotional codes via the Seller Dashboard, subject to platform approval.

To prevent abuse, each promotional code has predefined usage limits, expiration dates, and eligibility requirements. Shoplite regularly runs seasonal campaigns, such as Black Friday and holiday sales, with extensive marketing support.

Promotions are an effective way to drive sales volume, reduce inventory, and reward loyal customers.
"""
    },

      {
        "id": "doc17",
        "title": "Shoplite Seller Analytics and Reporting",
        "content": """Sellers on Shoplite gain access to a comprehensive analytics suite. Key metrics include sales performance, traffic sources, conversion rates, and customer demographics. Reports can be filtered by time range, product category, or marketing campaign.

Shoplite’s dashboard visualizes data with charts and graphs, enabling sellers to quickly identify trends. Advanced features include cohort analysis, abandoned cart tracking, and ROI measurement for paid advertisements.

Export tools allow sellers to download reports in CSV or Excel formats. API integration supports automated reporting for larger businesses. By providing data-driven insights, Shoplite empowers sellers to make informed decisions that optimize operations and maximize revenue.

"""
    },

    {
        "id": "doc18",
        "title": "Shoplite Fraud Prevention and Account Security",
        "content": """Shoplite employs multi-layered fraud prevention measures to protect buyers and sellers. Machine learning models analyze transaction patterns to detect unusual behavior, such as repeated failed login attempts or sudden spikes in high-value orders.

Accounts flagged for suspicious activity may be temporarily restricted until verification is complete. Verification can include identity confirmation, proof of payment method ownership, or additional documentation for sellers.

Two-factor authentication is encouraged for all users and mandatory for sellers handling high transaction volumes. Login notifications alert users of new device sign-ins, adding another layer of transparency.

By proactively detecting and addressing fraud risks, Shoplite maintains a secure and trustworthy marketplace.
"""
    },







]


### Prompts **(RGC)**

In [4]:
PROMPTS = {
    "base_retrieval_prompt": {
        "role": "You are a helpful Shoplite assistant.",
        "goal": "Provide accurate answers using only the provided Shoplite documentation.",
        "context_guidelines": [
            "Use only information from provided document snippets",
            "Cite specific documents when possible"
        ],
        "response_format": "Answer: [response]\nSources: [documents]"
    },

    "multi_doc_prompt": {
        "role": "You are a Shoplite support expert skilled at combining multiple sources.",
        "goal": "Synthesize answers that require retrieving and merging information from two or more documents.",
        "context_guidelines": [
            "Clearly indicate when multiple documents contribute to the answer",
            "Merge information into a single, concise explanation",
            "Always cite all relevant document titles"
        ],
        "response_format": "Answer: [integrated response]\nSources: [doc1, doc2, ...]"
    },

    "no_context_prompt": {
        "role": "You are a strict Shoplite knowledge base assistant.",
        "goal": "Refuse to answer if no relevant context is retrieved.",
        "context_guidelines": [
            "If no relevant documents are found, do not speculate",
            "Clearly tell the user that the answer is not available in the Shoplite documentation",
            "Encourage the user to rephrase or contact customer support if needed"
        ],
        "response_format": "Answer: Sorry, I couldn’t find relevant information in the documentation.\nSources: None"
    },

    "clarification_prompt": {
        "role": "You are a careful Shoplite assistant.",
        "goal": "Ask clarifying questions when user queries are vague, ambiguous, or incomplete.",
        "context_guidelines": [
            "If the query could match multiple features (e.g., 'payment options'), request clarification",
            "Provide examples of possible interpretations",
            "Only answer once the query is clarified"
        ],
        "response_format": "Clarification needed: [ask follow-up question]\nPossible interpretations: [list]"
    }
}


## Embeddings + FAISS Index

In [5]:
MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"

try:
    print(f"Loading {MODEL_NAME} in 4-bit…")
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
    )
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        quantization_config=quant_config,
    )
    generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        device_map="auto",
        max_new_tokens=256,
        do_sample=True,
        temperature=0.3,
        top_p=0.9,
    )
    print("Loaded Qwen2.5-7B-Instruct ")
except Exception as e:
    print("Error:", e)


    print("oaded TinyLlama fallback. No HF token needed.")

Loading Qwen/Qwen2.5-7B-Instruct in 4-bit…


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Device set to use cuda:0


✅ Loaded Qwen2.5-7B-Instruct (4-bit). No HF token needed.


## Load LLM

In [18]:
EMBEDDER_NAME = "all-MiniLM-L6-v2"
embedder = SentenceTransformer(EMBEDDER_NAME)

def create_embeddings(docs):

    texts = [d["content"] for d in docs]
    embeddings = embedder.encode(texts, normalize_embeddings=True)
    return np.asarray(embeddings, dtype=np.float32)

# Build FAISS index
doc_embeddings = create_embeddings(KNOWLEDGE_BASE)
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(doc_embeddings)

def retrieve_docs(query: str, top_k: int = 3, min_score: float = 0.45):
    query_emb = embedder.encode([query], normalize_embeddings=True)
    D, I = index.search(query_emb, top_k)
    results = []
    for score, idx in zip(D[0], I[0]):
        if idx == -1:
            continue
        if score < min_score:
            continue
        results.append({
            "rank": len(results)+1,
            "id": KNOWLEDGE_BASE[idx]["id"],
            "title": KNOWLEDGE_BASE[idx]["title"],
            "content": KNOWLEDGE_BASE[idx]["content"],
            "score": float(score),
        })
    return results




def generate_response(query: str, prompt_key: str = "base_retrieval_prompt", top_k: int = 3, max_new_tokens: int = 150):

    retrieved = retrieve_docs(query, top_k=top_k)


    if not retrieved and prompt_key != "no_context_refusal_prompt":
        prompt_key = "no_context_refusal_prompt"

    prompt = build_prompt(prompt_key, query, retrieved)


    raw = generator(prompt, max_new_tokens=max_new_tokens)[0]["generated_text"]

    answer = raw.split("Answer:", 1)[-1].strip() if "Answer:" in raw else raw.strip()

    return {
        "model": MODEL_NAME,
        "prompt_used": prompt_key,
        "response": answer,
        "sources": [r["title"] for r in retrieved]
    }


## Retrieval + Generation Functions

In [19]:
ngrok_token = input("Enter your ngrok token: ").strip()
if not ngrok_token:
    raise ValueError("ngrok token is required. Please re-run and enter your token.")

ngrok.set_auth_token(ngrok_token)

app = Flask(__name__)

@app.route("/")
def home():
    return "Shoplite RAG API is running!", 200

@app.route("/health")
def health_check():

    return jsonify({"status": "healthy"}), 200

@app.route("/ping", methods=["POST"])
def ping():
    data = request.json
    query = data.get("query", "default ping message")
    response = generator(query, max_new_tokens=50)[0]["generated_text"]

    response = response.replace(query, "").strip()
    return jsonify({"response": response}), 200

@app.route("/chat", methods=["POST"])
def chat():
    data = request.json
    query = data.get("query")
    prompt_key = data.get("prompt_key", "base_retrieval_prompt")
    top_k = data.get("top_k", 3)

    if not query:
        return jsonify({"error": "No query provided"}), 400

    rag_response = generate_response(query, prompt_key, top_k)


    if rag_response["prompt_used"] == "no_context_prompt":

         output = rag_response["response"]
         sources = []
    elif rag_response["prompt_used"] == "clarification_prompt":
        output = rag_response["response"]
        sources = rag_response["sources"]
    else:

        output = f"Answer: {rag_response['response']}"
        sources = rag_response["sources"]


    return jsonify({
        "query": query,
        "model": rag_response["model"],
        "prompt_used": rag_response["prompt_used"],
        "answer": output,
        "sources": sources
    }), 200

def run_app():
    app.run(host="0.0.0.0", port=5000, debug=False, use_reloader=False)

server_thread = threading.Thread(target=run_app, daemon=True)
server_thread.start()


time.sleep(2)
public_url = ngrok.connect(5000, "http")
print("Your public URL:", public_url.public_url)

Enter your ngrok token: 33LBjr4TVTl1yur4MM1Ck02bgzK_7XPhzxHmQP4bhegCBuXe5
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


Your public URL: https://collectedly-intercuspidal-allegra.ngrok-free.dev


In [20]:
import requests

BASE = public_url.public_url

print("Health ->", requests.get(f"{BASE}/health").json())

print("Ping ->", requests.post(f"{BASE}/ping", json={
    "query": "Say hello in one short sentence."
}).json())

print("Chat (RAG) ->", requests.post(f"{BASE}/chat", json={
    "query": "What payment methods are supported on Shoplite?",
    "prompt_key": "base_retrieval_prompt",
    "top_k": 1
}).json())

INFO:werkzeug:127.0.0.1 - - [01/Oct/2025 16:26:01] "GET /health HTTP/1.1" 200 -


Health -> {'status': 'healthy'}


INFO:werkzeug:127.0.0.1 - - [01/Oct/2025 16:26:03] "POST /ping HTTP/1.1" 200 -


Ping -> {'response': 'Hello! How can I assist you today?'}


INFO:werkzeug:127.0.0.1 - - [01/Oct/2025 16:26:12] "POST /chat HTTP/1.1" 200 -


Chat (RAG) -> {'answer': 'Answer: [response]\nSources: [documents] According to the provided document snippet on Shoplite Payment Methods and Security, the following payment methods are supported on Shoplite:\n\n- Major credit cards\n- Debit cards\n- PayPal\n- Apple Pay\n- Google Pay\n- Shoplite Wallet\n\nFor select regions, cash on delivery is also available. \n\nSources: Shoplite Payment Methods and Security document. []', 'model': 'Qwen/Qwen2.5-7B-Instruct', 'prompt_used': 'base_retrieval_prompt', 'query': 'What payment methods are supported on Shoplite?', 'sources': ['Shoplite Payment Methods and Security']}


In [21]:
import requests

BASE = public_url.public_url


TEST_CASES = [

    {"id": "H01", "query": "Does Shoplite support PayPal payments?",
     "must_include": ["PayPal", "supported"], "forbid": ["not", "refusal"]},
    {"id": "H02", "query": "Can I get free international shipping on Shoplite?",
     "must_include": ["not", "international"], "forbid": ["yes", "available", "free shipping"]}

]


def run_tests():
    results = []
    for case in TEST_CASES:
        resp = requests.post(f"{BASE}/chat", json={
            "query": case["query"],
            "prompt_key": "base_retrieval_prompt",
            "top_k": 3
        }).json()


        answer = resp.get("answer") or resp.get("response", "")
        answer_lower = answer.lower()

        passed = all(w.lower() in answer_lower for w in case["must_include"]) \
                 and all(f.lower() not in answer_lower for f in case["forbid"])

        results.append({
            "id": case["id"],
            "query": case["query"],
            "answer": answer[:200] + ("..." if len(answer) > 200 else ""),
            "pass": passed
        })
    return results



test_results = run_tests()
for r in test_results:
    status = "PASS" if r["pass"] else "FAIL"
    print(f"{r['id']} | {status}\nQ: {r['query']}\nA: {r['answer']}\n")

INFO:werkzeug:127.0.0.1 - - [01/Oct/2025 16:26:25] "POST /chat HTTP/1.1" 200 -
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
INFO:werkzeug:127.0.0.1 - - [01/Oct/2025 16:26:38] "POST /chat HTTP/1.1" 200 -


H01 | PASS
Q: Does Shoplite support PayPal payments?
A: Answer: [response]
Sources: [documents] Yes, Shoplite supports PayPal payments as one of the available payment methods for buyers. According to the "Shoplite Payment Methods and Security" document, cu...

H02 | FAIL
Q: Can I get free international shipping on Shoplite?
A: Answer: [response]
Sources: [documents] Based on the provided document snippet about Shoplite Order Tracking and Delivery, there is no mention of free international shipping being offered by Shoplite....



## Testing

In [22]:
import requests

BASE_URL = "http://localhost:5000"

question = "How do I create a seller account on Shoplite?"
response = requests.post(f"{BASE_URL}/chat", json={"query": question})

print(response.json())


INFO:werkzeug:127.0.0.1 - - [01/Oct/2025 16:27:02] "POST /chat HTTP/1.1" 200 -


{'answer': 'Answer: [response]\nSources: [documents] To create a seller account on Shoplite, you need to follow these steps:\n\n1. **Visit the Registration Page**: Start by going to the Shoplite registration page either through the website or mobile app.\n2. **Provide Required Information**: Submit your personal details such as full name, phone number, and a valid email address. Additionally, you will need to set up a secure password.\n3. **Submit Business Information**: Unlike regular user accounts, seller accounts require more detailed business information, including your company name, tax identification number, and proof of registration. This information will be subject to manual verification.\n4. **Confirm Email Verification**: After submission, Shoplite will send a verification email to the provided email address. You must confirm this email within', 'model': 'Qwen/Qwen2.5-7B-Instruct', 'prompt_used': 'base_retrieval_prompt', 'query': 'How do I create a seller account on Shoplite?

### Evals

In [35]:
GROUND_TRUTH_QA = [
    {
        "id": "Q01",
        "question": "How do I create a seller account on Shoplite?",
        "expected_context": ["doc9"],
        "authoritative_answer": "To create a seller account, visit the Shoplite seller registration page, provide business information including tax ID, and complete the verification process which takes 2–3 business days.",
        "required_keywords": ["seller registration", "business verification", "2–3 business days"],
        "forbidden_content": ["instant approval", "no verification required", "personal accounts"]
    },
    {
        "id": "Q02",
        "question": "What are Shoplite's return policies and how do I track my order status?",
        "expected_context": ["doc7", "doc6"],
        "authoritative_answer": "Shoplite allows most items to be returned within a 30-day window if unused and in original packaging. To initiate a return, log into your account and request a return authorization. Buyers receive a tracking number for their order, providing real-time updates including dispatch and estimated delivery.",
        "required_keywords": ["30-day return window", "return authorization", "order tracking"],
        "forbidden_content": ["no returns accepted", "lifetime returns"]
    },
    {
        "id": "Q03",
        "question": "Which payment methods are available for buyers on Shoplite?",
        "expected_context": ["doc5"],
        "authoritative_answer": "Buyers can pay with credit/debit cards, PayPal, Apple Pay, Google Pay, Shoplite Wallet, or cash on delivery in select regions. All transactions are encrypted and monitored for fraud.",
        "required_keywords": ["credit card", "PayPal", "encryption"],
        "forbidden_content": ["cash only", "unsecured payments"]
    },
    {
        "id": "Q04",
        "question": "How can buyers apply promotional codes during checkout?",
        "expected_context": ["doc2", "doc16"],
        "authoritative_answer": "Customers enter valid promotional codes at checkout. Discounts apply automatically to eligible items or categories. Each code has usage limits and expiration dates.",
        "required_keywords": ["promotional code", "discount applied", "usage limits"],
        "forbidden_content": ["manual discount", "codes without restrictions"]
    },
    {
        "id": "Q05",
        "question": "How can sellers manage inventory across multiple locations?",
        "expected_context": ["doc10"],
        "authoritative_answer": "Sellers can track stock across warehouses, update inventory automatically as sales occur, receive low-stock alerts, and integrate third-party inventory systems via API or CSV uploads.",
        "required_keywords": ["multi-location", "low-stock alerts", "API integration"],
        "forbidden_content": ["manual tracking only", "no alerts"]
    },
    {
        "id": "Q06",
        "question": "What features does the Shoplite mobile app provide to buyers and sellers?",
        "expected_context": ["doc13"],
        "authoritative_answer": "The mobile app offers personalized feeds, voice search, barcode scanning, push notifications, offline browsing, biometric login, and seller account management on the go.",
        "required_keywords": ["voice search", "offline browsing", "biometric login"],
        "forbidden_content": ["desktop features only", "no notifications"]
    },
    {
        "id": "Q07",
        "question": "How does Shoplite protect sensitive user data?",
        "expected_context": ["doc15"],
        "authoritative_answer": "Shoplite encrypts all data, hashes or tokenizes sensitive information, and complies with GDPR and CCPA. Users can download or delete their data.",
        "required_keywords": ["encryption", "GDPR", "data control"],
        "forbidden_content": ["selling data", "unencrypted storage"]
    },
    {
        "id": "Q08",
        "question": "What steps should a buyer take if their delivery is delayed?",
        "expected_context": ["doc6", "doc12"],
        "authoritative_answer": "Buyers should contact customer support. Shoplite mediates with couriers and may provide compensation or replacements for delayed or lost items.",
        "required_keywords": ["customer support", "delivery delay", "resolution"],
        "forbidden_content": ["no support for delays", "automatic compensation only"]
    },
    {
        "id": "Q09",
        "question": "How does Shoplite detect and prevent fraudulent transactions?",
        "expected_context": ["doc17", "doc5"],
        "authoritative_answer": "Machine learning monitors transaction patterns for anomalies. Suspicious accounts may be temporarily restricted. Two-factor authentication is recommended for buyers and required for high-volume sellers.",
        "required_keywords": ["fraud detection", "anomalies", "two-factor authentication"],
        "forbidden_content": ["no fraud checks", "manual detection only"]
    },
    {
        "id": "Q10",
        "question": "How does Shoplite calculate commissions for sellers?",
        "expected_context": ["doc11"],
        "authoritative_answer": "Shoplite charges a 10% commission on most completed sales. Fees vary by category, payment method, and optional services such as sponsored products or premium storefronts.",
        "required_keywords": ["commission", "transaction fees", "optional services"],
        "forbidden_content": ["no fees", "fixed flat rate"]
    },
    {
        "id": "Q11",
        "question": "How can a buyer initiate a return for damaged items?",
        "expected_context": ["doc7", "doc12"],
        "authoritative_answer": "Buyers log into their account, select the order, request a return authorization, print a prepaid shipping label, and ship the item. Support may assist if there are issues.",
        "required_keywords": ["return authorization", "prepaid label", "customer support"],
        "forbidden_content": ["no returns accepted", "instant refunds without inspection"]
    },
    {
        "id": "Q12",
        "question": "What analytics tools are available for sellers to track performance?",
        "expected_context": ["doc17"],
        "authoritative_answer": "Sellers can monitor sales, traffic sources, conversion rates, and customer demographics. Reports include charts, cohort analysis, abandoned cart tracking, and ROI for marketing campaigns.",
        "required_keywords": ["sales metrics", "conversion rates", "ROI"],
        "forbidden_content": ["manual tracking only", "no analytics"]
    },
    {
        "id": "Q13",
        "question": "How does Shoplite ensure secure checkout for all users?",
        "expected_context": ["doc4", "doc5"],
        "authoritative_answer": "Checkout is secured via SSL encryption. Buyers select payment methods and confirm order details. Sensitive payment information is protected and not stored directly on Shoplite servers.",
        "required_keywords": ["SSL encryption", "secure checkout", "payment protection"],
        "forbidden_content": ["unencrypted payment", "automatic charges without consent"]
    },
    {
        "id": "Q14",
        "question": "How can sellers create promotional campaigns?",
        "expected_context": ["doc16", "doc9"],
        "authoritative_answer": "Sellers can create discount codes, BOGO offers, or free shipping promotions via the Seller Dashboard. Codes require Shoplite approval and have usage limits and expiration dates.",
        "required_keywords": ["promotional code", "seller dashboard", "approval required"],
        "forbidden_content": ["automatic code creation", "unlimited discounts"]
    },
    {
        "id": "Q15",
        "question": "How does Shoplite moderate product reviews?",
        "expected_context": ["doc8"],
        "authoritative_answer": "Reviews are screened for inappropriate content, spam, and fake submissions. Verified purchase labels confirm authenticity. Sellers can respond to reviews.",
        "required_keywords": ["review moderation", "verified purchase", "seller response"],
        "forbidden_content": ["fake reviews allowed", "no moderation"]
    }
]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### Checking retrieval

In [56]:
def evaluate_rag_model(GROUND_TRUTH_QA, rag_pipeline):
    results = []
    for item in GROUND_TRUTH_QA:
        question = item['question']
        required_keywords = item.get('required_keywords', [])
        forbidden_terms = item.get('forbidden_content', [])

        # Generate response from RAG model
        response = rag_pipeline(question)

        # Check required keywords
        keywords_present = all(str(kw) in response for kw in required_keywords)
        forbidden_absent = all(str(ft) not in response for ft in forbidden_terms)

        passed = keywords_present and forbidden_absent
        results.append({
            "id": item["id"],
            "question": question,
            "response": response,
            "passed": passed
        })
    return results
    for r in results:
        print(f"Question {r['id']}: {'PASSED' if r['passed'] else 'FAILED'}")
        print(f"Response: {r['response']}\n")
evaluate_rag_model(GROUND_TRUTH_QA, generate_response)


[{'id': 'Q01',
  'question': 'How do I create a seller account on Shoplite?',
  'response': {'answer': "You are a helpful Shoplite assistant.\nAnswer the user's question based on the context below.\n\nContext:\nSellers who want to join Shoplite must first create a seller account by visiting the dedicated registration page. They are required to submit business information, such as company name, tax identification, and proof of registration. Shoplite verifies these documents within 2–3 business days.\n\nOnce approved, sellers can access the Seller Dashboard, which includes tools for product uploads, inventory tracking, and sales analytics. They can also manage orders, returns, and customer messages directly from the dashboard. Shoplite provides training materials and best-practice guides to help sellers succeed.\n\nSellers must comply with Shoplite’s marketplace policies, which include accurate product descriptions, timely shipping, and responsible customer service. Accounts that violate

## Multi-Trun Conversation chat interface

In [29]:
def generate_response(query):

    retrieved = retrieve_docs(query)


    context = "\n".join([doc["content"] for doc in retrieved])
    prompt = f"You are a helpful Shoplite assistant.\nAnswer the user's question based on the context below.\n\nContext:\n{context}\n\nQuestion: {query}\nAnswer:"

    inputs = tokenizer(prompt, return_tensors="pt")


    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}


    outputs = model.generate(**inputs, max_new_tokens=200)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return {"answer": answer, "sources": [doc["id"] for doc in retrieved]}


In [33]:
from rich.console import Console
from rich.markdown import Markdown
from rich.panel import Panel

console = Console()

def chat_with_model():
    console.print(Panel("[bold cyan]Shoplite Assistant[/] (type 'exit' to quit)", expand=False))

    while True:
        user_input = console.input("[bold green]You:[/] ")
        if user_input.lower() in ["exit", "quit", "bye"]:
            console.print("[bold cyan]Assistant:[/] Goodbye!")
            break

        try:
            response = generate_response(user_input)
            answer = response.get("answer", "No answer generated.")
            sources = response.get("sources", [])


            console.print(Markdown(f"**Assistant:** {answer}"))

            if sources:
                console.print(Panel(f"Sources: {', '.join(sources)}", style="cyan"))
        except Exception as e:
            console.print(f"[red] Error:[/] {str(e)}")

chat_with_model()


╭─────────────────────────────────────────────╮
│ 🤖 Shoplite Assistant (type 'exit' to quit) │
╰─────────────────────────────────────────────╯

You:

hello]


Assistant: You are a helpful Shoplite assistant. Answer the user's question based on the context below.            

Context:                                                                                                           

Question: hello] Answer: Hello! How can I assist you today? ]

You:

...

You:

whats the capital of france


Assistant: You are a helpful Shoplite assistant. Answer the user's question based on the context below.            

Context:                                                                                                           

Question: whats the capital of france Answer: The capital of France is Paris.                                      

To provide more context, Paris is not only the capital but also the most populous city in France. It serves as the 
country's cultural, commercial, and political center. Paris is renowned for its iconic landmarks such as the Eiffel
Tower, Louvre Museum, and Notre-Dame Cathedral, making it one of the world's major global cities. Additionally, it 
is known for its fashion, art, and cuisine, attracting millions of tourists annually. []

You:

KeyboardInterrupt: Interrupted by user